In [1]:
%load_ext autoreload
%autoreload 2

from spot.utils import proj_root, os
os.chdir(proj_root())

In [2]:
bad_code = """
def parse_qualified_name(tree: ast.Attribute):
    segs = []
    while isinstance(tree, ast.Attribute):
        segs.append(tree.attr)
        tree = tree.value  # type: ignore
    assert isinstance(tree, ast.Name)
    segs.append(tree.id)
    return tuple(reversed(segs))
"""
from spot.experiments.type4py import cst, remove_newer_syntax

bad_code = remove_newer_syntax(cst.parse_module(bad_code)).code
print(bad_code)



from typing import Any, List, Tuple, Dict, Set, Union, Optional
def parse_qualified_name(tree: ast.Attribute):
    segs = []
    while isinstance(tree, ast.Attribute):
        segs.append(tree.attr)
        tree = tree.value  # type: ignore
    assert isinstance(tree, ast.Name)
    segs.append(tree.id)
    return tuple(reversed(segs))



In [3]:
import requests
from spot.utils import proj_root, read_file

bad_code = """
def parse_qualified_name(tree: ast.Attribute):
    segs = []
    while isinstance(tree, ast.Attribute):
        segs.append(tree.attr)
        tree = tree.value  # type: ignore
    assert isinstance(tree, ast.Name)
    segs.append(tree.id)
    return tuple(reversed(segs))
"""

r = requests.post("https://type4py.com/api/predict?tc=0", bad_code)
r.json()


{'error': None,
 'response': {'classes': [],
  'funcs': [{'docstring': {'func': None, 'long_descr': None, 'ret': None},
    'fn_lc': [[2, 0], [9, 32]],
    'fn_var_ln': {'segs': [[3, 4], [3, 8]], 'tree': [[6, 8], [6, 12]]},
    'fn_var_occur': {'segs': [['segs', 'append', 'tree', 'attr'],
      ['segs', 'append', 'tree', 'id'],
      ['tuple', 'reversed', 'segs']],
     'tree': [['isinstance', 'tree', 'ast', 'Attribute'],
      ['segs', 'append', 'tree', 'attr'],
      ['tree', 'tree', 'value'],
      ['isinstance', 'tree', 'ast', 'Name'],
      ['segs', 'append', 'tree', 'id']]},
    'name': 'parse_qualified_name',
    'params': {'tree': 'ast.Attribute'},
    'params_descr': {'tree': ''},
    'params_occur': {'tree': [['isinstance', 'tree', 'ast', 'Attribute'],
      ['segs', 'append', 'tree', 'attr'],
      ['tree', 'tree', 'value'],
      ['isinstance', 'tree', 'ast', 'Name'],
      ['segs', 'append', 'tree', 'id']]},
    'params_p': {'args': [], 'kwargs': [], 'tree': []},
    'q_na

In [4]:
from spot.experiments.type4py import Type4PyResponseParser
from spot.static_analysis import FunctionSignature

pred_signatures = Type4PyResponseParser("spot.model").parse(r.json())
for path, sig in pred_signatures.items():
    if isinstance(sig, FunctionSignature):
        print(path)
        print("\t" + str(sig))


spot.model/parse_qualified_name
	FuncSig(() -> int)


In [2]:
from spot.static_analysis import PythonProject
from spot.utils import *
from spot.model import ModelWrapper
from spot.visualization import pretty_print_dict, assert_eq
from spot.experiments.type4py import eval_type4py_on_projects
from spot.function_dataset import data_project_from_dir


# dataset_name = "ManyTypes4Py"
dataset_name = "InferTypes4Py"

# test_projects = [PythonProject.from_root(proj_root(), ignore_dirs={".venv", "data"})]

repos_dir = get_dataset_dir(dataset_name) / "repos" / "test"
test_repo_paths = [f for f in repos_dir.iterdir() if f.is_dir()]
test_projects = pmap(
    data_project_from_dir,
    test_repo_paths,
    desc="Loading test projects",
)
eval_r = eval_type4py_on_projects(test_projects, max_workers=4)


Calling Type4Py: 100%|██████████| 128/128 [01:26<00:00,  1.48it/s]


In [4]:
from spot.static_analysis import SignatureErrorAnalysis, AccuracyMetric

common_names = ModelWrapper.load_common_type_names(
    get_model_dir() / "model-v7--TrainingConfig(drop_env_types=False)"
)
metrics = AccuracyMetric.default_metrics(common_type_names=common_names)
# acc_metric = AccuracyMetric(common_type_names=ubiq_names)

accs = {
    m.name: SignatureErrorAnalysis(
        eval_r.pred_maps,
        eval_r.label_maps,
        m,
        error_on_mismatched_signature=False,
    ).accuracies
    for m in metrics
}
pretty_print_dict(accs)


plain_acc:
   plain_acc: 22.63% (count=2.5k)
   plain_acc_by_common:
      rare: 0.12% (count=813)
      common: 33.33% (count=1.7k)
   plain_acc_by_cat:
      FuncArg: 19.89% (count=1.6k)
      FuncReturn: 33.12% (count=640)
      ClassAtribute: 12.50% (count=240)
      GlobalVar: 75.00% (count=4)
   plain_acc_label_size: 1.7436
   plain_acc_pred_size: 1.2612
   plain_acc_ignored_labels: 0
   n_skipped_types: 23
   n_missing_types: 113
acc:
   acc: 20.86% (count=2.4k)
   acc_by_common:
      rare: 0.12% (count=813)
      common: 31.76% (count=1.5k)
   acc_by_cat:
      FuncArg: 21.87% (count=1.5k)
      FuncReturn: 21.76% (count=602)
      ClassAtribute: 12.18% (count=238)
      GlobalVar: 0.00% (count=1)
   acc_label_size: 1.6719
   acc_pred_size: 1.181
   acc_ignored_labels: 164
   n_skipped_types: 23
   n_missing_types: 113
base_acc:
   base_acc: 22.64% (count=2.4k)
   base_acc_by_common:
      rare: 0.17% (count=572)
      common: 29.83% (count=1.8k)
   base_acc_by_cat:
      Func